# <center> Task 1

Some of the Product_URLs in the log file might have been corrupted. Write a Python (or PySpark) procedure to determine which Product_URLs are corrupted. Let us assume that if a Product_url in the log file doesn’t occur in the products table, it is regarded as corrupted. Using this procedure identify and list the corrupted URLs.  (10) 

# <center> Answer 

- We can utilize python and pandas package to deal with these problems. pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,built on top of the Python programming language.First, we use pd.merge (here I set the parameter "how = left",which means this function uses only keys from left frame, similar to a SQL left outer join; preserve key order) function to merge the log file and the products file. As a result, final dataframe has 216 missing values, which means there are 216 corrupted URLs. And I list the 216 corrupted URLs.

In [1]:
import pandas as pd
import urllib.parse
import warnings
import difflib
df_log = pd.read_csv('log2.csv')
df_product = pd.read_csv('products.csv')
df_prodect_category = pd.read_csv('product_categories.csv')

In [2]:
df_product = df_product.drop_duplicates(subset=['Product URL'])

In [3]:
df_merge = pd.merge(df_log, df_product, on="Product URL", how="left",validate='many_to_one')
df_merge['product'].isnull().sum()

216

In [4]:
df_corrupted = df_merge[df_merge.isna().T.any()]
df_corrupted['Product URL']

83      https://haier.com/refrigermtors
109        https://sony.comftelevisions
123              https://lg.com/gashers
171              https://leks.com/jeans
203      https://InstantPot.con/cookers
                     ...               
9729           http://nejoK.co/blenders
9773         https://maytag.cpm/washers
9845         https://guessmcom/perfumes
9881    https://samsuag.com/televisions
9951     https://InstantPotycom/cookers
Name: Product URL, Length: 216, dtype: object

In [5]:
df_product.shape

(50, 3)

# <center> Task 2

For each corrupted URL what will you do with it? Don’t assume that for each corrupted URL the correct approach is to delete that log entry. What if the URL contained ‘.cam’ instead of ‘.com’ but otherwise corresponded with a URL in the ‘products’ table? In that case the proper approach would be to correct the URL. In other cases, the URL might be so corrupted that the best approach would be to delete that log entry (the entire row). Describe your approach to dealing with corrupted URLs. That is, describe your approach to determining that a URL is too corrupted to be rescued. It must describe a) a procedure for determining the degree to which the URL is corrupted, b) a threshold for determining in terms of this degree of corruption whether it can be corrected, and c) for those which can be corrected, identifying its corrected form. For extra credit implement this in a Python (or PySpark) program. (25 + 20 points for extra-credit) 

# <center> Answer

- Correcting 216 corrupted URLs one by one is a difficult and tedious task. I wrote an algorithm to help me with this challenge. 
    - First, I utilized urllib.parse.quote function to replace special characters in URL string using the %xx escape. Thus, I got 216 strings without special characters. 
    - Second, I used difflib.SequenceMatcher function (This function quantifies the degree of similarity between two strings) to compare 216 corrupted URLs and 50 correct URLS. If similarity between two strings > 0.94, which means that the corrupted URL found its correct counterpart, I will directly replace the corrupted URL with its correct counterpart.
- Eventually, this algorithm successfully corrected the 216 corrupted URLs. And I list the 10 correct URLs (There are 10,000 correct URLs in total).

In [6]:
df_merge = df_merge.dropna()

In [7]:
def string_similar(s1, s2):
    return difflib.SequenceMatcher(None, s1, s2).quick_ratio()

In [8]:
for i in range(df_corrupted['product'].isnull().sum()):
    for j in range(50):
        if string_similar(urllib.parse.quote(df_corrupted.iloc[i,2]),urllib.parse.quote(df_product.iloc[j,1]))>0.94:
            df_corrupted.iloc[i,2] = df_product.iloc[j,1]

In [9]:
x=df_corrupted.iloc[:,0:6]
df_merge1 = pd.merge(x, df_product, on="Product URL", how="left",validate='many_to_one')
df_real = pd.concat([df_merge, df_merge1])
df_real.head(10)

,Sentiment,Publication_URL,Product URL,clickORnot,gender,age_group,product,product_type
0,positive,https://www.foxnews.com/,https://lees.com/jeans,0,female,juvenile,Lee jeans,jeans
1,neutral,https://www.mirror.co.uk/news/,https://coach.com/purses,0,male,young,Coach purse,women's purse
2,negative,https://www.nbcnews.com/,https://covergirl.co/lipsticks,0,male,middle-age,covergirl lipstick,lipstick
3,positive,https://www.examiner.com/,https://covergirl.co/makeup,0,male,juvenile,Covergirl makeup,makeup
4,negative,https://www.nj.com,https://dell.com/computers,1,female,young,Dell computer,computer
5,positive,https://www.chicagotribune.com/,https://remington.com/shavers,1,male,senior,Remington shaver,shaver
6,negative,https://www.msn.com/en-us/news,https://dell.com/laptops,0,male,young,Dell laptop,computer
7,positive,https://www.salon.com/,https://samsung.com/washers,1,male,juvenile,Samsung washer,washer
8,positive,https://www.cnet.com/,https://Ikea.com/sofas,0,male,senior,Ikea sofa,furniture
9,negative,https://www.buzzfeed.com/,https://InstantPot.com/cookers,0,female,senior,InstantPot pressure cooker,pressure cooker


# <center> Task 3

For each product, compute all the Publication_URLs containing an ad for that product. (Don’t just give the results. Show all the work by which you got those results. This applies to all the questions below.) (10) 

# <center> Answer

- Once we have the correct dataframe without missing value, the computation of subsequent tasks is feasible. For Task 3, groupby.sum() function in Pandas package could compute and list all the Publication_URLs containing an ad for that product. (groupby 'product' and 'Publication_URL').

In [10]:
warnings.filterwarnings("ignore")
df_product.sort_values("product",inplace=True,ignore_index = True)
pro = df_product["product"].drop_duplicates().reset_index(drop=True)
df_real_q3 = df_real.iloc[:,[6,1]]
df_real_q3.sort_values("product",inplace=True,ignore_index =True)
df_q3 = pd.DataFrame(columns=['product','Publication_URL'])
for i in range(50):
    df = df_real_q3[df_real_q3['product'] == pro[i]]
    y = ""
    for j in range(df.shape[0]):
        y += df_real_q3.iloc[j,1]
        y +=","
    df_q3= df_q3.append(pd.DataFrame({'product':[pro[i]],'Publication_URL':[y]}),ignore_index = True)
df_q3

,product,Publication_URL
0,Apple computer,"https://www.mirror.co.uk/news/,https://www.bos..."
1,Apple iPad,"https://www.mirror.co.uk/news/,https://www.bos..."
2,Apple laptop,"https://www.mirror.co.uk/news/,https://www.bos..."
3,BasilBasel perfume,"https://www.mirror.co.uk/news/,https://www.bos..."
4,Broyhill recliner,"https://www.mirror.co.uk/news/,https://www.bos..."
5,Centrum MultiVitamins,"https://www.mirror.co.uk/news/,https://www.bos..."
6,Clinique moisturizer,"https://www.mirror.co.uk/news/,https://www.bos..."
7,Coach purse,"https://www.mirror.co.uk/news/,https://www.bos..."
8,Cougar jeans,"https://www.mirror.co.uk/news/,https://www.bos..."
9,Covergirl makeup,"https://www.mirror.co.uk/news/,https://www.bos..."


# <center> Task 4

For each product type, compute all the Publication_URLs containing an ad for that product type. Your solution must be scalable. That is, it should work well even if there are hundreds of products in each product_type and there are hundreds of product_types. (Hint: To make it scalable you should consider using a Python or PySpark script instead of a SQL query.) (20)

# <center> Answer

- For Task 4, groupby.sum() function in Pandas package could compute and list all the Publication_URLs containing an ad for that product type. (groupby 'product_type' and 'Publication_URL'). And this solution is scalable. It should work well even if there are hundreds of products in each product_type and there are hundreds of product_types.

In [11]:
warnings.filterwarnings("ignore")
df_product.sort_values("product_type",inplace=True,ignore_index = True)
x = df_product["product_type"].drop_duplicates().reset_index(drop=True)
df_real_q4 = df_real.iloc[:,[7,1]]
df_real_q4.sort_values("product_type",inplace=True,ignore_index =True)
df_q5 = pd.DataFrame(columns=['product_type','Publication_URL'])
for i in range(24):
    df = df_real_q4[df_real_q4['product_type'] == x[i]]
    y = ""
    for j in range(df.shape[0]):
        y += df_real_q4.iloc[j,1]
        y +=","
    df_q5= df_q5.append(pd.DataFrame({'product_type':[x[i]],'Publication_URL':[y]}),ignore_index = True)
df_q5

,product_type,Publication_URL
0,blender,"https://techcrunch.com/,https://www.npr.org/,h..."
1,car,"https://techcrunch.com/,https://www.npr.org/,h..."
2,coffee,"https://techcrunch.com/,https://www.npr.org/,h..."
3,computer,"https://techcrunch.com/,https://www.npr.org/,h..."
4,dryer,"https://techcrunch.com/,https://www.npr.org/,h..."
5,elliptical trainer,"https://techcrunch.com/,https://www.npr.org/,h..."
6,face cream,"https://techcrunch.com/,https://www.npr.org/,h..."
7,furniture,"https://techcrunch.com/,https://www.npr.org/,h..."
8,jeans,"https://techcrunch.com/,https://www.npr.org/,h..."
9,lipstick,"https://techcrunch.com/,https://www.npr.org/,h..."


# <center> Task 5

Save this information in the database. Should you save it in the products table or the product_categories table or should you create a new table, product_type_pubURLs, and save this information in this table? If you create a new table, make sure to set up all the appropriate foreign key constraints. On the other hand, if you use one of the existing tables, explain how you will avoid redundancy in your data. In either case, justify your decision. (10)

# <center> Answer

- For task 5, I choose to create create a new table product_type_pubURLs. This new table has two attributes: "product_type" and "Publication_URL". And I merge the result into the following form and store it this way in sql:
    - "type_a" "[url1, url2, url3, url4, ...]"
- And I set up the appropriate foreign key constraints (foreign key: product_type_pubURLs (product_type) to product_categories (product_type)) 

In [12]:
import os
import mysql.connector

In [13]:
db = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="001011",
    database="data1050"   
)
print(db)
if db.is_connected():
    print("CONNECTION SUCCESSFUL")

CONNECTION SUCCESSFUL


In [14]:
mycursor = db.cursor()
mycursor.execute("DROP TABLE IF EXISTS `product_type_sentiment_clickrate`")
mycursor.execute("DROP TABLE IF EXISTS `product_type_pubURLs`")
mycursor.execute("DROP TABLE IF EXISTS `product_categories`")

In [15]:
#df_prodect_category
df_prodect_category = pd.read_csv('product_categories.csv')
df_prodect_category.sort_values("product_type",inplace=True,ignore_index = True)
df_prodect_category=df_prodect_category.drop(labels=17)
mycursor.execute(
"""
CREATE TABLE product_categories (
product_type varchar(255) PRIMARY KEY,
category varchar(255)
)
"""
)
mycursor.execute("DESCRIBE product_categories")
for x in mycursor:
    print(x)

('product_type', b'varchar(255)', 'NO', 'PRI', None, '')
('category', b'varchar(255)', 'YES', '', None, '')


In [16]:
for i in range(24):
    product_type = df_prodect_category.iloc[i,0]
    category = df_prodect_category.iloc[i,1]
    insert_val = tuple((product_type,category))
    mycursor.execute("INSERT INTO product_categories(product_type,category) VALUES (%s, %s)", insert_val)
mycursor.execute("select * from product_categories")
mycursor.fetchall()

[('blender', 'small kitchen appliances'),
 ('car', 'transportation'),
 ('coffee', 'packaged food'),
 ('computer', 'consumer electronics'),
 ('dryer', 'large kitchen appliances'),
 ('elliptical trainer', 'fitness equipment'),
 ('face cream', 'beauty products'),
 ('furniture', 'household durables'),
 ('jeans', 'apparel'),
 ('lipstick', 'beauty products'),
 ('makeup', 'beauty products'),
 ('pants', 'apparel'),
 ('perfume', 'beauty products'),
 ('pressure cooker', 'small kitchen appliances'),
 ('refrigerator', 'large kitchen appliances'),
 ('rowing machine', 'fitness equipment'),
 ('shaver', 'consumer electronics'),
 ('speakers', 'consumer electronics'),
 ('tablet', 'consumer electronics'),
 ('television', 'consumer electronics'),
 ('treadmill', 'fitness equipment'),
 ('vitamin', 'health'),
 ('washer', 'large kitchen appliances'),
 ("women's purse", 'accessories')]

In [17]:
mycursor.execute(
"""
CREATE TABLE product_type_pubURLs (
product_type varchar(255) PRIMARY KEY,
Publication_URL text,
FOREIGN KEY (product_type) REFERENCES product_categories(product_type)
)
"""
)
mycursor.execute("DESCRIBE product_type_pubURLs")
for x in mycursor:
    print(x)

('product_type', b'varchar(255)', 'NO', 'PRI', None, '')
('Publication_URL', b'text', 'YES', '', None, '')


In [18]:
df_product.sort_values("product_type",inplace=True,ignore_index = True)
x = df_product["product_type"].drop_duplicates().reset_index(drop=True)

In [19]:
warnings.filterwarnings("ignore")
for i in range(24):
    product_type = df_q5.iloc[i,0]
    Publication_URL = df_q5.iloc[i,1]
    insert_val = tuple((product_type,Publication_URL))
    mycursor.execute("INSERT INTO product_type_pubURLs(product_type,Publication_URL) VALUES (%s, %s)", insert_val)

In [20]:
mycursor = db.cursor()
mycursor.execute("select * from product_type_pubURLs LIMIT 0,1")
mycursor.fetchall()

[('blender',
  'https://techcrunch.com/,https://www.npr.org/,https://www.usatoday.com/,https://www.vox.com/,https://www.boston.com,https://www.foxnews.com/,https://www.bostonglobe.com/,https://www.examiner.com/,https://mashable.com/,https://www.theatlantic.com/,https://www.nydailynews.com/,https://www.independent.co.uk/,https://www.dallasnews.com/,https://www.sfgate.com/,https://abcnews.go.com/,https://time.com/,https://www.cbsnews.com/,https://www.vox.com/,https://www.sfgate.com/,https://www.huffingtonpost.com/,https://www.cbsnews.com/,https://www.usatoday.com/,https://www.examiner.com/,https://www.cnn.com/,https://www.foxnews.com/,https://www.foxnews.com/,https://nypost.com/,https://www.al.com/,https://www.upworthy.com/,https://www.businessinsider.com/,https://www.nbcnews.com/,https://www.nbcnews.com/,https://nypost.com/,https://www.dailymail.co.uk/,https://mashable.com/,https://www.vox.com/,https://time.com/,https://www.npr.org/,https://www.dallasnews.com/,https://www.usnews.com/,ht

In [21]:
db.close()
mycursor.close()

True

# <center> Task 6

For each product, compute the click rate for it. (Click rate is the number of times a display of an ad was clicked on (by any user) divided by the number of times it was displayed (to any user). That is, the click rate is not specific to each user.) 

# <center> Answer

- For Task 6, groupby.apply(lambda x: x.sum()/len(x)) function in Pandas package could compute and list all the click rate for each product. (groupby 'product'). And this solution is scalable. It should work well even if there are hundreds of products in each product_type and there are hundreds of product_types.

In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
import numpy as np
df_prod = df_real[["product"]]
df_click = df_real[["clickORnot"]]

In [23]:
onehot_ftrs = ['product']
preprocessor = ColumnTransformer(
transformers=[('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'),onehot_ftrs)])
clf = Pipeline(steps=[('preprocessor', preprocessor)])
df_prod = clf.fit_transform(df_prod)
df_train = pd.DataFrame(df_prod, columns=preprocessor.get_feature_names_out())[:len(df_prod)]
df_prod_click = pd.concat([df_train,pd.DataFrame(np.reshape(np.array(df_click), (1, -1)).ravel(),columns=['clickORnot'])],axis=1)

In [24]:
click_rate = np.zeros(50)
df_product.sort_values("product",inplace=True,ignore_index = True)
for i in range(50):
    click_rate[i] = df_prod_click.loc[df_prod_click[df_prod_click.columns.values[i]] == 1]['clickORnot'].sum()/df_prod_click.loc[df_prod_click[df_prod_click.columns.values[i]] == 1][df_prod_click.columns.values[i]].sum()
    #print('The Click rate for', df_product['product'][i],'is', click_rate[i])

In [25]:
df_log_products_on_product_URL_6 = df_real[['product', 'clickORnot']]
df_log_products_on_product_URL_6.groupby(['product']).apply(lambda x: x.sum()/len(x))

,clickORnot
product,
Apple computer,0.793103
Apple iPad,0.503788
Apple laptop,0.560976
BasilBasel perfume,0.649351
Broyhill recliner,0.539216
Centrum MultiVitamins,0.626556
Clinique moisturizer,0.805556
Coach purse,0.388646
Cougar jeans,0.260073


# <center> Task 7

For each product, compute the click rate for each sentiment type. (10) 

# <center> Answer

- For each product, groupby.apply(lambda x: x.sum()/len(x)) function in Pandas package could compute and list all the click rate for each sentiment type. (groupby 'product','Sentiment'). And this solution is scalable. It should work well even if there are hundreds of products in each product_type and there are hundreds of product_types.

In [26]:
df_senti = df_real[["Sentiment"]]
df_senti = df_senti.reset_index(drop=True)
df_prod_click = df_prod_click.reset_index(drop=True)

In [27]:
df_prod_click_sentiment = pd.concat([df_prod_click,df_senti],axis=1)

In [28]:
positive = df_prod_click_sentiment[df_prod_click_sentiment['Sentiment']== 'positive'] 
neutral = df_prod_click_sentiment[df_prod_click_sentiment['Sentiment']== 'neutral'] 
negative = df_prod_click_sentiment[df_prod_click_sentiment['Sentiment']== 'negative'] 

In [29]:
click_rate_positive = np.zeros(50)
click_rate_neutral = np.zeros(50)
click_rate_negative = np.zeros(50)
df_product.sort_values("product",inplace=True,ignore_index = True)
for i in range(50):
    click_rate_positive[i] = positive.loc[positive[positive.columns.values[i]] == 1]['clickORnot'].sum()/positive.loc[positive[positive.columns.values[i]] == 1][positive.columns.values[i]].sum()
    click_rate_neutral[i] = neutral.loc[neutral[neutral.columns.values[i]] == 1]['clickORnot'].sum()/neutral.loc[neutral[neutral.columns.values[i]] == 1][neutral.columns.values[i]].sum()
    click_rate_negative[i] = negative.loc[negative[negative.columns.values[i]] == 1]['clickORnot'].sum()/negative.loc[negative[negative.columns.values[i]] == 1][negative.columns.values[i]].sum()
    #print('The Click rate for', df_product['product'][i],'(Sentiment is positive)','is', click_rate_positive[i])
    #print('The Click rate for', df_product['product'][i],'(Sentiment is neutral)','is', click_rate_neutral[i])
    #print('The Click rate for', df_product['product'][i],'(Sentiment is negative)','is', click_rate_negative[i])

In [30]:
pd.set_option('display.max_rows', 1000)
df_log_products_on_product_URL_7 = df_real[['product', 'Sentiment', 'clickORnot']]
df_log_products_on_product_URL_7.groupby(['product', 'Sentiment']).apply(lambda x: x.sum()/len(x))

clickORnot
product                    Sentiment            
Apple computer             negative     0.700000
                           neutral      0.923077
                           positive     0.764706
Apple iPad                 negative     0.391304
                           neutral      0.627907
                           positive     0.500000
Apple laptop               negative     0.153846
                           neutral      0.725000
                           positive     0.772727
BasilBasel perfume         negative     0.771429
                           neutral      0.864407
                           positive     0.366667
Broyhill recliner          negative     0.814286
                           neutral      0.562500
                           positive     0.242857
Centrum MultiVitamins      negative     0.835294
                           neutral      0.814815
                           positive     0.186667
Clinique moisturizer       negative     0.902778
                           neutral      0.930556
                           positive     0.583333
Coach purse                negative     0.316456
                           neutral      0.453333
                           positive     0.400000
Cougar jeans               negative     0.083333
                           neutral      0.311688
                           positive     0.390000
Covergirl makeup           negative     0.112676
                           neutral      0.384615
                           positive     0.245283
Dell computer              negative     0.830986
                           neutral      0.382353
                           positive     0.719512
Dell laptop                negative     0.145299
                           neutral      0.357143
                           positive     0.452830
Docker pants               negative     0.835821
                           neutral      0.818182
                           positive     0.333333
Ford  sedan                negative     0.013158
                           neutral      0.125000
                           positive     0.265823
Gillette shaver            negative     0.904762
                           neutral      0.960784
                           positive     0.139535
Giorgio perfume            negative     0.638554
                           neutral      0.969697
                           positive     0.830769
Givenchy perfume           negative     0.757576
                           neutral      0.302632
                           positive     0.347826
Guess perfume              negative     0.172414
                           neutral      0.306452
                           positive     0.788462
Haier refrigerator         negative     0.173077
                           neutral      0.166667
                           positive     0.283019
Hamilton Beach blender     negative     0.671642
                           neutral      0.353846
                           positive     0.216216
Ikea sofa                  negative     0.840000
                           neutral      0.594203
                           positive     0.322034
InstantPot pressure cooker negative     0.000000
                           neutral      0.864407
                           positive     0.714286
Jaguar perfume             negative     0.434783
                           neutral      0.700000
                           positive     0.311111
Kaai handbags              negative     0.490566
                           neutral      0.956522
                           positive     0.512821
LG TV                      negative     0.298507
                           neutral      0.590909
                           positive     0.644444
LG dryer                   negative     0.680851
                           neutral      0.791667
                           positive     0.395349
LG washer                  negative     0.153846
                           neutral      0.516667
                    

# <center> Task8

For each product type, compute the click rate for it. (10) 

# <center> Answer

- For Task 8, groupby.apply(lambda x: x.sum()/len(x)) function in Pandas package could compute and list all the click rate for each product type. (groupby 'product'). And this solution is scalable. It should work well even if there are hundreds of products in each product_type and there are hundreds of product_types.

In [31]:
df_prod_type = df_real[["product_type"]]
df_click = df_real[["clickORnot"]]

In [32]:
onehot_ftrs = ['product_type']
preprocessor = ColumnTransformer(
transformers=[('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'),onehot_ftrs)])
clf = Pipeline(steps=[('preprocessor', preprocessor)])
df_prod_type = clf.fit_transform(df_prod_type)
df_train = pd.DataFrame(df_prod_type, columns=preprocessor.get_feature_names_out())[:len(df_prod_type)]
df_train = df_train.reset_index(drop=True)
df_click = df_click.reset_index(drop=True)
df_prod_type_click = pd.concat([df_train,df_click],axis=1)

In [33]:
click_rate = np.zeros(24)
df_product.sort_values("product_type",inplace=True,ignore_index = True)
x = df_product["product_type"].drop_duplicates().reset_index(drop=True)
for i in range(24):
    click_rate[i] = df_prod_type_click.loc[df_prod_type_click[df_prod_type_click.columns.values[i]] == 1]['clickORnot'].sum()/df_prod_type_click.loc[df_prod_type_click[df_prod_type_click.columns.values[i]] == 1][df_prod_type_click.columns.values[i]].sum()
    #print('The Click rate for', x[i],'is', click_rate[i])

In [34]:
df_log_products_on_product_URL_8 = df_real[['product_type', 'clickORnot']]
df_log_products_on_product_URL_8.groupby(['product_type']).apply(lambda x: x.sum()/len(x))

,clickORnot
product_type,
blender,0.499237
car,0.370450
coffee,0.355294
computer,0.558583
dryer,0.452716
elliptical trainer,0.528409
face cream,0.805556
furniture,0.554974
jeans,0.451477


# <center> Task9

For each product type compute the click rate for each sentiment type. (10) 

# <center> Answer

- For each product type, groupby.apply(lambda x: x.sum()/len(x)) function in Pandas package could compute and list all the click rate for each sentiment type. (groupby 'product_type','Sentiment'). And this solution is scalable. It should work well even if there are hundreds of products in each product_type and there are hundreds of product_types.

In [35]:
df_prod_type_click_sentiment = pd.concat([df_prod_type_click,df_senti],axis=1)

In [36]:
positive = df_prod_type_click_sentiment[df_prod_type_click_sentiment['Sentiment']== 'positive'] 
neutral = df_prod_type_click_sentiment[df_prod_type_click_sentiment['Sentiment']== 'neutral'] 
negative = df_prod_type_click_sentiment[df_prod_type_click_sentiment['Sentiment']== 'negative'] 

In [37]:
warnings.filterwarnings("ignore")
click_rate_positive = np.zeros(24)
click_rate_neutral = np.zeros(24)
click_rate_negative = np.zeros(24)

df_9_positive = pd.DataFrame(columns=['type','Sentiment', 'click rate'])
df_9_neutral = pd.DataFrame(columns=['type','Sentiment', 'click rate'])
df_9_negative = pd.DataFrame(columns=['type', 'Sentiment','click rate'])

for i in range(24):
    click_rate_positive[i] = positive.loc[positive[positive.columns.values[i]]== 1]['clickORnot'].sum()/positive.loc[positive[positive.columns.values[i]]== 1][positive.columns.values[i]].sum()
    click_rate_neutral[i] = neutral.loc[neutral[neutral.columns.values[i]] ==1]['clickORnot'].sum()/neutral.loc[neutral[neutral.columns.values[i]] ==1][neutral.columns.values[i]].sum()
    click_rate_negative[i] = negative.loc[negative[negative.columns.values[i]]== 1]['clickORnot'].sum()/negative.loc[negative[negative.columns.values[i]]== 1][negative.columns.values[i]].sum()
for i in range(24):
    df_9_positive = df_9_positive.append(pd.DataFrame({'type':[x[i]],'Sentiment':['positive'],'click rate':[click_rate_positive[i]]}),ignore_index = True)
    df_9_neutral= df_9_neutral.append(pd.DataFrame({'type':[x[i]],'Sentiment':['neutral'],'click rate':[click_rate_neutral[i]]}),ignore_index = True)
    df_9_negative= df_9_negative.append(pd.DataFrame({'type':[x[i]],'Sentiment':['negative'],'click rate':[click_rate_negative[i]]}),ignore_index = True)
df_9 = pd.concat([df_9_positive, df_9_neutral,df_9_negative])
df_9.sort_values("type",inplace=True,ignore_index = True)

In [38]:
df_10 = pd.DataFrame(columns=['type', 'negative_click_rate','neutral_click_rate','positive_click_rate'])
for i in range(24):
    df_10 = df_10.append(pd.DataFrame({'type':[x[i]],'negative_click_rate':[click_rate_negative[i]],'neutral_click_rate':[click_rate_neutral[i]],'positive_click_rate':[click_rate_positive[i]]}),ignore_index = True)
df_10  

,type,negative_click_rate,neutral_click_rate,positive_click_rate
0,blender,0.742739,0.337963,0.378788
1,car,0.391892,0.581699,0.156627
2,coffee,0.465278,0.357143,0.241135
3,computer,0.501355,0.570270,0.604972
4,dryer,0.323699,0.734568,0.308642
5,elliptical trainer,0.491228,0.737705,0.344828
6,face cream,0.902778,0.930556,0.583333
7,furniture,0.825000,0.578947,0.279070
8,jeans,0.264822,0.609756,0.509881
9,lipstick,0.692308,0.511450,0.825688


# <center> Task 10

Save this information you computed in 9 above in a database table. Should you save it in the products table or the product_categories table or the product_type_pubURLs table, or should you create a new table product_type_sentiment_clickrate, and save this information in this table? If you create a new table, make sure to set up all the appropriate foreign key constraints. On the other hand, if you use one of the existing tables, explain how you will avoid redundancy in your data. In either case, justify your decision.  (10)

# <center> Answer

- For task 10, I choose to create create a new table product_type_sentiment_clickrate. This new table has four attributes: "product_type", "negative click rate", "neutral click rate" and "positive click rate". And I merge the result into the following form and store it this way in sql:
    - "type_a" "negative click rate" "neutral click rate" "positive click rate"
- And I set up the appropriate foreign key constraints (foreign key: product_type_sentiment_clickrate (product_type) to product_categories (product))

In [39]:
df_10 = pd.DataFrame(columns=['type', 'negative_click_rate','neutral_click_rate','positive_click_rate'])
for i in range(24):
    df_10 = df_10.append(pd.DataFrame({'type':[x[i]],'negative_click_rate':[click_rate_negative[i]],'neutral_click_rate':[click_rate_neutral[i]],'positive_click_rate':[click_rate_positive[i]]}),ignore_index = True)
df_10    

,type,negative_click_rate,neutral_click_rate,positive_click_rate
0,blender,0.742739,0.337963,0.378788
1,car,0.391892,0.581699,0.156627
2,coffee,0.465278,0.357143,0.241135
3,computer,0.501355,0.570270,0.604972
4,dryer,0.323699,0.734568,0.308642
5,elliptical trainer,0.491228,0.737705,0.344828
6,face cream,0.902778,0.930556,0.583333
7,furniture,0.825000,0.578947,0.279070
8,jeans,0.264822,0.609756,0.509881
9,lipstick,0.692308,0.511450,0.825688


In [40]:
db = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="001011",
    database="data1050"   
)
print(db)
if db.is_connected():
    print("CONNECTION SUCCESSFUL")

CONNECTION SUCCESSFUL


In [41]:
mycursor = db.cursor()
mycursor.execute("DROP TABLE IF EXISTS `product_type_sentiment_clickrate`")

mycursor.execute(
"""
CREATE TABLE product_type_sentiment_clickrate (
product_type varchar(255) PRIMARY KEY,
negative_click_rate FLOAT(4),
neutral_click_rate FLOAT(4),
positive_click_rate FLOAT(4),
FOREIGN KEY (product_type) REFERENCES product_categories(product_type)
)
"""
)
mycursor.execute("DESCRIBE product_type_sentiment_clickrate")
for x in mycursor:
    print(x)

('product_type', b'varchar(255)', 'NO', 'PRI', None, '')
('negative_click_rate', b'float', 'YES', '', None, '')
('neutral_click_rate', b'float', 'YES', '', None, '')
('positive_click_rate', b'float', 'YES', '', None, '')


In [42]:
for i in range(24):
    product_type = df_10.iloc[i,0]
    negative_click_rate = df_10.iloc[i,1]
    neutral_click_rate = df_10.iloc[i,2]
    positive_click_rate = df_10.iloc[i,3]
    insert_val = tuple((product_type,negative_click_rate,neutral_click_rate,positive_click_rate))
    mycursor.execute("INSERT INTO product_type_sentiment_clickrate(product_type,negative_click_rate,neutral_click_rate,positive_click_rate) VALUES (%s, %s,%s, %s)", insert_val)

In [43]:
mycursor = db.cursor()
mycursor.execute("select * from product_type_sentiment_clickrate")
mycursor.fetchall()

[('blender', 0.742739, 0.337963, 0.378788),
 ('car', 0.391892, 0.581699, 0.156627),
 ('coffee', 0.465278, 0.357143, 0.241135),
 ('computer', 0.501355, 0.57027, 0.604972),
 ('dryer', 0.323699, 0.734568, 0.308642),
 ('elliptical trainer', 0.491228, 0.737705, 0.344828),
 ('face cream', 0.902778, 0.930556, 0.583333),
 ('furniture', 0.825, 0.578947, 0.27907),
 ('jeans', 0.264822, 0.609756, 0.509881),
 ('lipstick', 0.692308, 0.51145, 0.825688),
 ('makeup', 0.112676, 0.384615, 0.245283),
 ('pants', 0.835821, 0.818182, 0.333333),
 ('perfume', 0.555556, 0.610561, 0.532646),
 ('pressure cooker', 0.0, 0.864407, 0.714286),
 ('refrigerator', 0.122222, 0.388889, 0.347368),
 ('rowing machine', 0.2, 0.142857, 0.297297),
 ('shaver', 0.528, 0.651685, 0.44186),
 ('speakers', 0.348837, 0.705882, 0.543478),
 ('tablet', 0.391304, 0.627907, 0.5),
 ('television', 0.319277, 0.701987, 0.592814),
 ('treadmill', 0.316456, 0.58427, 0.56),
 ('vitamin', 0.835294, 0.814815, 0.186667),
 ('washer', 0.376984, 0.518987, 

In [44]:
db.close()
mycursor.close()

True

# <center> Task 11

Determine if the gender of the person viewing ads make a difference with regard to the click rate of ads shown in different sentiment context. That is, determine if there are any ‘significant’ differences in the correlation between the sentiment type of the ad context and clicking on the product type conditioned on gender. You can decide if any difference counts as ‘significant’. (This is not a yes or no question. Compute the different correlations.)  (10) 

# <center> Answer

- How to decide if any difference counts as ‘significant'?
    - For a specific product type and a specific sentiment type, we can claculate the click rate for each gender (eg. for blender and sentiment type is positive, we get click rate (male) = 0.380435 and click rate (female) = 0.377358),	
    - I define the "degree of difference" as: 
## <center>degree of difference $= \frac{|clickrate\ (male)\ -\ clickrate\ (female)|}{clickrate\ (male)\ +\ clickrate\ (female)} $</center>
    - if the "degree of difference" $> 0.05$, I define the difference between click rate (male) and click rate (female) as "significant" (eg. for blender and sentiment type is positive, we get click rate (male) = 0.380435 and click rate (female) = 0.377358, 
    <center>"degree of difference" $= \frac{|0.380435-0.377358|}{(0.380435+0.377358)} < 0.05$</center>, so its "not significant").
- And I compute and list didifferent correlations.

In [45]:
df_gender = df_real[["gender"]]
df_gender = df_gender.reset_index(drop=True)

In [46]:
df_male_positive = pd.DataFrame(columns=['type','Sentiment', 'click rate (male)'])
df_male_neutral = pd.DataFrame(columns=['type','Sentiment', 'click rate (male)'])
df_male_negative = pd.DataFrame(columns=['type', 'Sentiment','click rate (male)'])
df_prod_type_click_sentiment_gender = pd.concat([df_prod_type_click_sentiment,df_gender],axis=1)
x = df_product["product_type"].drop_duplicates().reset_index(drop=True)
df_diff_male =df_prod_type_click_sentiment_gender.loc[df_prod_type_click_sentiment_gender['gender'] == 'male']


positive = df_diff_male[df_diff_male['Sentiment']== 'positive'] 
neutral = df_diff_male[df_diff_male['Sentiment']== 'neutral'] 
negative = df_diff_male[df_diff_male['Sentiment']== 'negative'] 

warnings.simplefilter(action='ignore', category=FutureWarning)

click_rate_positive = np.zeros(24)
click_rate_neutral = np.zeros(24)
click_rate_negative = np.zeros(24)
for i in range(24):
    click_rate_positive[i] = positive.loc[positive[positive.columns.values[i]] == 1]['clickORnot'].sum()/positive.loc[positive[positive.columns.values[i]] == 1][positive.columns.values[i]].sum()
    click_rate_neutral[i] = neutral.loc[neutral[neutral.columns.values[i]] == 1]['clickORnot'].sum()/neutral.loc[neutral[neutral.columns.values[i]] == 1][neutral.columns.values[i]].sum()
    click_rate_negative[i] = negative.loc[negative[negative.columns.values[i]] == 1]['clickORnot'].sum()/negative.loc[negative[negative.columns.values[i]] == 1][negative.columns.values[i]].sum()
for i in range(24):
    df_male_positive = df_male_positive.append(pd.DataFrame({'type':[x[i]],'Sentiment':['positive'],'click rate (male)':[click_rate_positive[i]]}),ignore_index = True)
    df_male_neutral= df_male_neutral.append(pd.DataFrame({'type':[x[i]],'Sentiment':['neutral'],'click rate (male)':[click_rate_neutral[i]]}),ignore_index = True)
    df_male_negative= df_male_negative.append(pd.DataFrame({'type':[x[i]],'Sentiment':['negative'],'click rate (male)':[click_rate_negative[i]]}),ignore_index = True)
df_male = pd.concat([df_male_positive, df_male_neutral,df_male_negative])

In [47]:
df_female_positive = pd.DataFrame(columns=['type','Sentiment', 'click rate (female)'])
df_female_neutral = pd.DataFrame(columns=['type','Sentiment', 'click rate (female)'])
df_female_negative = pd.DataFrame(columns=['type', 'Sentiment','click rate (female)'])
df_prod_type_click_sentiment_gender = pd.concat([df_prod_type_click_sentiment,df_gender],axis=1)
df_diff_female =df_prod_type_click_sentiment_gender.loc[df_prod_type_click_sentiment_gender['gender'] == 'female']


positive = df_diff_female[df_diff_female['Sentiment']== 'positive'] 
neutral = df_diff_female[df_diff_female['Sentiment']== 'neutral'] 
negative = df_diff_female[df_diff_female['Sentiment']== 'negative'] 

warnings.simplefilter(action='ignore', category=FutureWarning)

click_rate_positive = np.zeros(24)
click_rate_neutral = np.zeros(24)
click_rate_negative = np.zeros(24)
for i in range(24):
    click_rate_positive[i] = positive.loc[positive[positive.columns.values[i]] == 1]['clickORnot'].sum()/positive.loc[positive[positive.columns.values[i]] == 1][positive.columns.values[i]].sum()
    click_rate_neutral[i] = neutral.loc[neutral[neutral.columns.values[i]] == 1]['clickORnot'].sum()/neutral.loc[neutral[neutral.columns.values[i]] == 1][neutral.columns.values[i]].sum()
    click_rate_negative[i] = negative.loc[negative[negative.columns.values[i]] == 1]['clickORnot'].sum()/negative.loc[negative[negative.columns.values[i]] == 1][negative.columns.values[i]].sum()
for i in range(24):
    df_female_positive = df_female_positive.append(pd.DataFrame({'type':[x[i]],'Sentiment':['positive'],'click rate (female)':[click_rate_positive[i]]}),ignore_index = True)
    df_female_neutral= df_female_neutral.append(pd.DataFrame({'type':[x[i]],'Sentiment':['neutral'],'click rate (female)':[click_rate_neutral[i]]}),ignore_index = True)
    df_female_negative= df_female_negative.append(pd.DataFrame({'type':[x[i]],'Sentiment':['negative'],'click rate (female)':[click_rate_negative[i]]}),ignore_index = True)
df_female = pd.concat([df_female_positive, df_female_neutral,df_female_negative])

In [48]:
df_merge_gender = pd.merge(df_male, df_female, on=['type','Sentiment'], how="left",validate='one_to_one')
df_gender_diff = pd.DataFrame(columns=['diff'])
for i in range(72):
    if abs(df_merge_gender.iloc[i,2] - df_merge_gender.iloc[i,3])/(df_merge_gender.iloc[i,2] + df_merge_gender.iloc[i,3]+0.00001) > 0.05:
        df_gender_diff = df_gender_diff.append(pd.DataFrame({'diff':['significant']}),ignore_index = True)
    else:
        df_gender_diff = df_gender_diff.append(pd.DataFrame({'diff':['not']}),ignore_index = True)
df_merge_gender_diff = pd.concat([df_merge_gender, df_gender_diff],axis = 1)
df_merge_gender_diff

,type,Sentiment,click rate (male),click rate (female),diff
0,blender,positive,0.380435,0.377358,not
1,car,positive,0.183908,0.126582,significant
2,coffee,positive,0.217949,0.269841,significant
3,computer,positive,0.614035,0.596859,not
4,dryer,positive,0.372093,0.236842,significant
5,elliptical trainer,positive,0.424242,0.240000,significant
6,face cream,positive,0.625000,0.550000,significant
7,furniture,positive,0.298507,0.258065,significant
8,jeans,positive,0.539130,0.485507,significant
9,lipstick,positive,0.866667,0.775510,significant


In [49]:
#y = df_merge_gender_diff.sort_values(by = ['type','Sentiment'],ignore_index = True)
#df_11 = df_real.iloc[:,[0,7]]
#x= df_11.groupby( ["product_type","Sentiment"]).count()
#x['result']=0
#for i in range(72):
#    x.iloc[i,0] = y['diff'][i]
#x

# <center> Task 12

The same question as 11 above but replace gender with age-group. (10) 

- How to decide if any difference counts as ‘significant'?
    - For a specific product type and a specific sentiment type, we can claculate the click rate for each age group 
    - I define the "degree of difference" as: $i = 1, 2, 3, 4$
## <center>degree of difference $= \frac{clickrate\ (age\ i)\ -\ average\ clickrate}{clickrate\ (age\ i)\ +\ average\ clickrate} $</center>
    - if the "degree of difference" $> 0.05$, I define that this age group is significantly higher; 
    - if the "degree of difference" $< 0.05$, I define that this age group is significantly lower. 
    - If there’s an age group that is significantly higher than the average and an age group that is significantly lower, it would be reasonable to conclude that age is a significant factor for that product type and that sentiment.
- And I compute and list didifferent correlations.
- (PS: 'juvenile’, 'young’, 'middle-age’, 'senior' can be encoded as 1, 2, 3, 4 (1 being juvenile and so on). )

In [50]:
df_age = df_real[["age_group"]]
df_age = df_age.reset_index(drop=True)
df_product.sort_values("product_type",inplace=True,ignore_index = True)
x = df_product["product_type"].drop_duplicates().reset_index(drop=True)

In [51]:
df_juvenile_positive = pd.DataFrame(columns=['type','Sentiment', 'click rate1'])
df_juvenile_neutral = pd.DataFrame(columns=['type','Sentiment', 'click rate1'])
df_juvenile_negative = pd.DataFrame(columns=['type', 'Sentiment','click rate1'])
df_prod_type_click_sentiment_age = pd.concat([df_prod_type_click_sentiment,df_age],axis=1)
df_diff_juvenile =df_prod_type_click_sentiment_age.loc[df_prod_type_click_sentiment_age['age_group'] == 'juvenile']


positive = df_diff_juvenile[df_diff_juvenile['Sentiment']== 'positive'] 
neutral = df_diff_juvenile[df_diff_juvenile['Sentiment']== 'neutral'] 
negative = df_diff_juvenile[df_diff_juvenile['Sentiment']== 'negative'] 

warnings.simplefilter(action='ignore', category=FutureWarning)

click_rate_positive = np.zeros(24)
click_rate_neutral = np.zeros(24)
click_rate_negative = np.zeros(24)
for i in range(24):
    click_rate_positive[i] = positive.loc[positive[positive.columns.values[i]] == 1]['clickORnot'].sum()/positive.loc[positive[positive.columns.values[i]] == 1][positive.columns.values[i]].sum()
    click_rate_neutral[i] = neutral.loc[neutral[neutral.columns.values[i]] == 1]['clickORnot'].sum()/neutral.loc[neutral[neutral.columns.values[i]] == 1][neutral.columns.values[i]].sum()
    click_rate_negative[i] = negative.loc[negative[negative.columns.values[i]] == 1]['clickORnot'].sum()/negative.loc[negative[negative.columns.values[i]] == 1][negative.columns.values[i]].sum()
for i in range(24):
    df_juvenile_positive = df_juvenile_positive.append(pd.DataFrame({'type':[x[i]],'Sentiment':['positive'],'click rate1':[click_rate_positive[i]]}),ignore_index = True)
    df_juvenile_neutral= df_juvenile_neutral.append(pd.DataFrame({'type':[x[i]],'Sentiment':['neutral'],'click rate1':[click_rate_neutral[i]]}),ignore_index = True)
    df_juvenile_negative= df_juvenile_negative.append(pd.DataFrame({'type':[x[i]],'Sentiment':['negative'],'click rate1':[click_rate_negative[i]]}),ignore_index = True)
df_juvenile = pd.concat([df_juvenile_positive, df_juvenile_neutral,df_juvenile_negative])

In [52]:
df_young_positive = pd.DataFrame(columns=['type','Sentiment', 'click rate2'])
df_young_neutral = pd.DataFrame(columns=['type','Sentiment', 'click rate2'])
df_young_negative = pd.DataFrame(columns=['type', 'Sentiment','click rate2'])
df_prod_type_click_sentiment_age = pd.concat([df_prod_type_click_sentiment,df_age],axis=1)
df_diff_young =df_prod_type_click_sentiment_age.loc[df_prod_type_click_sentiment_age['age_group'] == 'young']


positive = df_diff_young[df_diff_young['Sentiment']== 'positive'] 
neutral = df_diff_young[df_diff_young['Sentiment']== 'neutral'] 
negative = df_diff_young[df_diff_young['Sentiment']== 'negative'] 

warnings.simplefilter(action='ignore', category=FutureWarning)

click_rate_positive = np.zeros(24)
click_rate_neutral = np.zeros(24)
click_rate_negative = np.zeros(24)
for i in range(24):
    click_rate_positive[i] = positive.loc[positive[positive.columns.values[i]] == 1]['clickORnot'].sum()/positive.loc[positive[positive.columns.values[i]] == 1][positive.columns.values[i]].sum()
    click_rate_neutral[i] = neutral.loc[neutral[neutral.columns.values[i]] == 1]['clickORnot'].sum()/neutral.loc[neutral[neutral.columns.values[i]] == 1][neutral.columns.values[i]].sum()
    click_rate_negative[i] = negative.loc[negative[negative.columns.values[i]] == 1]['clickORnot'].sum()/negative.loc[negative[negative.columns.values[i]] == 1][negative.columns.values[i]].sum()
for i in range(24):
    df_young_positive = df_young_positive.append(pd.DataFrame({'type':[x[i]],'Sentiment':['positive'],'click rate2':[click_rate_positive[i]]}),ignore_index = True)
    df_young_neutral= df_young_neutral.append(pd.DataFrame({'type':[x[i]],'Sentiment':['neutral'],'click rate2':[click_rate_neutral[i]]}),ignore_index = True)
    df_young_negative= df_young_negative.append(pd.DataFrame({'type':[x[i]],'Sentiment':['negative'],'click rate2':[click_rate_negative[i]]}),ignore_index = True)
df_young = pd.concat([df_young_positive, df_young_neutral,df_young_negative])    

In [53]:
df_middleage_positive = pd.DataFrame(columns=['type','Sentiment', 'click rate3'])
df_middleage_neutral = pd.DataFrame(columns=['type','Sentiment', 'click rate3'])
df_middleage_negative = pd.DataFrame(columns=['type', 'Sentiment','click rate3'])
df_prod_type_click_sentiment_age = pd.concat([df_prod_type_click_sentiment,df_age],axis=1)
df_diff_middleage =df_prod_type_click_sentiment_age.loc[df_prod_type_click_sentiment_age['age_group'] == 'middle-age']


positive = df_diff_middleage[df_diff_middleage['Sentiment']== 'positive'] 
neutral = df_diff_middleage[df_diff_middleage['Sentiment']== 'neutral'] 
negative = df_diff_middleage[df_diff_middleage['Sentiment']== 'negative'] 

warnings.simplefilter(action='ignore', category=FutureWarning)

click_rate_positive = np.zeros(24)
click_rate_neutral = np.zeros(24)
click_rate_negative = np.zeros(24)
for i in range(24):
    click_rate_positive[i] = positive.loc[positive[positive.columns.values[i]] == 1]['clickORnot'].sum()/positive.loc[positive[positive.columns.values[i]] == 1][positive.columns.values[i]].sum()
    click_rate_neutral[i] = neutral.loc[neutral[neutral.columns.values[i]] == 1]['clickORnot'].sum()/neutral.loc[neutral[neutral.columns.values[i]] == 1][neutral.columns.values[i]].sum()
    click_rate_negative[i] = negative.loc[negative[negative.columns.values[i]] == 1]['clickORnot'].sum()/negative.loc[negative[negative.columns.values[i]] == 1][negative.columns.values[i]].sum()
for i in range(24):
    df_middleage_positive = df_middleage_positive.append(pd.DataFrame({'type':[x[i]],'Sentiment':['positive'],'click rate3':[click_rate_positive[i]]}),ignore_index = True)
    df_middleage_neutral= df_middleage_neutral.append(pd.DataFrame({'type':[x[i]],'Sentiment':['neutral'],'click rate3':[click_rate_neutral[i]]}),ignore_index = True)
    df_middleage_negative= df_middleage_negative.append(pd.DataFrame({'type':[x[i]],'Sentiment':['negative'],'click rate3':[click_rate_negative[i]]}),ignore_index = True)
df_middleage = pd.concat([df_middleage_positive, df_middleage_neutral,df_middleage_negative])

In [54]:
df_senior_positive = pd.DataFrame(columns=['type','Sentiment', 'click rate4'])
df_senior_neutral = pd.DataFrame(columns=['type','Sentiment', 'click rate4'])
df_senior_negative = pd.DataFrame(columns=['type', 'Sentiment','click rate4'])
df_prod_type_click_sentiment_age = pd.concat([df_prod_type_click_sentiment,df_age],axis=1)
df_diff_senior =df_prod_type_click_sentiment_age.loc[df_prod_type_click_sentiment_age['age_group'] == 'senior']


positive = df_diff_senior[df_diff_senior['Sentiment']== 'positive'] 
neutral = df_diff_senior[df_diff_senior['Sentiment']== 'neutral'] 
negative = df_diff_senior[df_diff_senior['Sentiment']== 'negative'] 

warnings.simplefilter(action='ignore', category=FutureWarning)

click_rate_positive = np.zeros(24)
click_rate_neutral = np.zeros(24)
click_rate_negative = np.zeros(24)
for i in range(24):
    click_rate_positive[i] = positive.loc[positive[positive.columns.values[i]] == 1]['clickORnot'].sum()/positive.loc[positive[positive.columns.values[i]] == 1][positive.columns.values[i]].sum()
    click_rate_neutral[i] = neutral.loc[neutral[neutral.columns.values[i]] == 1]['clickORnot'].sum()/neutral.loc[neutral[neutral.columns.values[i]] == 1][neutral.columns.values[i]].sum()
    click_rate_negative[i] = negative.loc[negative[negative.columns.values[i]] == 1]['clickORnot'].sum()/negative.loc[negative[negative.columns.values[i]] == 1][negative.columns.values[i]].sum()
for i in range(24):
    df_senior_positive = df_senior_positive.append(pd.DataFrame({'type':[x[i]],'Sentiment':['positive'],'click rate4':[click_rate_positive[i]]}),ignore_index = True)
    df_senior_neutral= df_senior_neutral.append(pd.DataFrame({'type':[x[i]],'Sentiment':['neutral'],'click rate4':[click_rate_neutral[i]]}),ignore_index = True)
    df_senior_negative= df_senior_negative.append(pd.DataFrame({'type':[x[i]],'Sentiment':['negative'],'click rate4':[click_rate_negative[i]]}),ignore_index = True)
df_senior = pd.concat([df_senior_positive, df_senior_neutral,df_senior_negative])

In [55]:
df_merge_age1 = pd.merge(df_juvenile,df_young, on=['type','Sentiment'], how="left",validate='one_to_one')
df_merge_age2 = pd.merge(df_merge_age1, df_middleage, on=['type','Sentiment'], how="left",validate='one_to_one')
df_merge_age = pd.merge(df_merge_age2, df_senior, on=['type','Sentiment'], how="left",validate='one_to_one')

In [56]:
df_merge_age['average_clickrate'] = 0
for i in range(72):
    df_merge_age['average_clickrate'][i] = (df_merge_age.iloc[i,2] + df_merge_age.iloc[i,3] + df_merge_age.iloc[i,4] + df_merge_age.iloc[i,5])/4

In [57]:
df_age_diff_1 = pd.DataFrame(columns=['diff 1'])
for i in range(72):
    if (df_merge_age.iloc[i,2] - df_merge_age.iloc[i,6])/(df_merge_age.iloc[i,2] + df_merge_age.iloc[i,6]) > 0.05:
        df_age_diff_1 = df_age_diff_1.append(pd.DataFrame({'diff 1':[1]}),ignore_index = True)
    elif (df_merge_age.iloc[i,2] - df_merge_age.iloc[i,6])/(df_merge_age.iloc[i,2] + df_merge_age.iloc[i,6]) < -0.05:
        df_age_diff_1 = df_age_diff_1.append(pd.DataFrame({'diff 1':[0.1]}),ignore_index = True)
    else:
        df_age_diff_1 = df_age_diff_1.append(pd.DataFrame({'diff 1':[0]}),ignore_index = True)

df_age_diff_2 = pd.DataFrame(columns=['diff 2'])
for i in range(72):
    if (df_merge_age.iloc[i,3] - df_merge_age.iloc[i,6])/(df_merge_age.iloc[i,3] + df_merge_age.iloc[i,6]) > 0.05:
        df_age_diff_2 = df_age_diff_2.append(pd.DataFrame({'diff 2':[1]}),ignore_index = True)
    elif (df_merge_age.iloc[i,3] - df_merge_age.iloc[i,6])/(df_merge_age.iloc[i,3] + df_merge_age.iloc[i,6]) < -0.05:
        df_age_diff_2 = df_age_diff_2.append(pd.DataFrame({'diff 2':[0.1]}),ignore_index = True)
    else:
        df_age_diff_2 = df_age_diff_2.append(pd.DataFrame({'diff 2':[0]}),ignore_index = True)

df_age_diff_3 = pd.DataFrame(columns=['diff 3'])
for i in range(72):
    if (df_merge_age.iloc[i,4] - df_merge_age.iloc[i,6])/(df_merge_age.iloc[i,4] + df_merge_age.iloc[i,6]) > 0.05:
        df_age_diff_3 = df_age_diff_3.append(pd.DataFrame({'diff 3':[1]}),ignore_index = True)
    elif (df_merge_age.iloc[i,4] - df_merge_age.iloc[i,6])/(df_merge_age.iloc[i,4] + df_merge_age.iloc[i,6]) < -0.05:
        df_age_diff_3 = df_age_diff_3.append(pd.DataFrame({'diff 3':[0.1]}),ignore_index = True)
    else:
        df_age_diff_3 = df_age_diff_3.append(pd.DataFrame({'diff 3':[0]}),ignore_index = True)
        
df_age_diff_4 = pd.DataFrame(columns=['diff 4'])
for i in range(72):
    if (df_merge_age.iloc[i,5] - df_merge_age.iloc[i,6])/(df_merge_age.iloc[i,5] + df_merge_age.iloc[i,6]) > 0.05:
        df_age_diff_4 = df_age_diff_4.append(pd.DataFrame({'diff 4':[1]}),ignore_index = True)
    elif (df_merge_age.iloc[i,5] - df_merge_age.iloc[i,6])/(df_merge_age.iloc[i,5] + df_merge_age.iloc[i,6]) < -0.05:
        df_age_diff_4 = df_age_diff_4.append(pd.DataFrame({'diff 4':[0.1]}),ignore_index = True)
    else:
        df_age_diff_4 = df_age_diff_4.append(pd.DataFrame({'diff 4':[0]}),ignore_index = True)
df_merge_age_diff_1 = pd.concat([df_age_diff_1, df_age_diff_2],axis = 1)
df_merge_age_diff_2 = pd.concat([df_merge_age_diff_1, df_age_diff_3],axis = 1)
df_merge_age_diff_3 = pd.concat([df_merge_age_diff_2, df_age_diff_4],axis = 1)
df_merge_age_diff_3['final'] = 0
for i in range(72):
    df_merge_age_diff_3['final'][i] = df_merge_age_diff_3.iloc[i,0] + df_merge_age_diff_3.iloc[i,1] + df_merge_age_diff_3.iloc[i,2] + df_merge_age_diff_3.iloc[i,3]
df_merge_age_diff_3
df_merge_age_diff_3['final sign'] = 0
for i in range(72):
    if df_merge_age_diff_3.iloc[i,4] >1 and df_merge_age_diff_3.iloc[i,4] % 1 >0:
        df_merge_age_diff_3['final sign'][i] = 'yes'
    else:
        df_merge_age_diff_3['final sign'][i] = 'no'
df_merge_age['significant'] = 0
for i in range(72):
    df_merge_age['significant'][i] = df_merge_age_diff_3.iloc[i,5]
df_merge_age

,type,Sentiment,click rate1,click rate2,click rate3,click rate4,average_clickrate,significant
0,blender,positive,0.320000,0.461538,0.297872,0.428571,0.376996,yes
1,car,positive,0.050000,0.187500,0.106383,0.322581,0.166616,yes
2,coffee,positive,0.048780,0.290323,0.361111,0.303030,0.250811,yes
3,computer,positive,0.517647,0.648936,0.557895,0.693182,0.604415,yes
4,dryer,positive,0.350000,0.348837,0.179487,0.350000,0.307081,yes
5,elliptical trainer,positive,0.411765,0.363636,0.277778,0.333333,0.346628,yes
6,face cream,positive,0.500000,0.571429,0.687500,0.588235,0.586791,yes
7,furniture,positive,0.121212,0.405405,0.343750,0.222222,0.273147,yes
8,jeans,positive,0.428571,0.558824,0.441176,0.606557,0.508782,yes
9,lipstick,positive,0.692308,0.870968,0.807692,0.923077,0.823511,yes


# <center> Task 13

Based on your results make your recommendations. These should be in the form:  
- a. Based on our analysis (give details of your analysis), ads for such and such product are most likely to produce clicks in such and sentiment context (or state that we see no correlation between click rate of an ad for a product and the sentiment context of the ad)  
- b. Based on our analysis (with details), ads for such and such product are most likely to produce clicks in such and sentiment context by viewers of such and such gender (or state that we see no correlation between click rate of an ad for a product and the sentiment context of the ad and the gender of the viewer). 
- c.  Based on our analysis (with details), ads for such and such product are most likely to produce clicks in such and sentiment context by viewers of such and such age-group (or state that we see no correlation between click rate of an ad for a product and the sentiment context of the ad and the age-group of the viewer).  (15) 

# <center> Answer

Prof. Pradhan said 'For question 13: The recommendations are meant to be for specific products (not product types). After all, the client wants to know how to spend the ad budget for specific products. But your analysis in questions 11 and 12 are for specific product types. Here you have to make the assumption that if the analysis applies to a product type, then it applies to each product under that product type. That is, a recommendation you would make for a product type would also apply to each product under that type, even if the click rates for the product type are not true for each product. In submitting your recommendations, in the interest of brevity you can choose just one product for each product type.'

# <center> Answer to 13(a)

## A recommendation you would make for a product type would also apply to each product under that type.---- Prof. Pradhan

- For a specific product type, we can claculate the click rate for each sentiment type 
    - I define the "degree of difference" as:
## <center>degree of difference $= \frac{|clickrate\ (type\ a)\ -\ average\ clickrate|}{clickrate\ (type\ a)\ +\ average\ clickrate} $</center>
- And then, for each product type, I add up the degree of difference of the three types of sentiment and pick the top ten product type. And  I choose just one product for each product type. 

### TOP 10
- InstantPot pressure cooker - ads for InstantPot pressure cooker are most likely to produce clicks in neutral and positive context - pressure cooker


- Centrum MultiVitamins - ads for Centrum MultiVitamins are most likely to produce clicks in negative and neutral context - vitamin


- Maytag refrigerator - ads for Maytag refrigerator are most likely to produce clicks in neutral context - refrigerator


- Tesla - ads for Tesla are most likely to produce clicks in neutral context - car


- Maytag dryer - ads for Maytag dryer are most likely to produce clicks in neutral context - dryer


- Covergirl makeup - ads for Covergirl makeup are most likely to produce clicks in neutral context - makeup


- Docker pants	- ads for Docker pants are most likely to produce clicks in negative and neutral context - pants


- Broyhill recliner - ads for Broyhill recliner are most likely to produce clicks in negative context - furniture


- NemoK blender - ads for NemoK blender are most likely to produce clicks in negative context - blender


- Lee jeans - ads for Lee jeans	 are most likely to produce clicks in neutral context - jeans

In [58]:
df_10['average_clickrate'] = 0
for i in range(24):
    df_10['average_clickrate'][i] = (df_10.iloc[i,1] + df_10.iloc[i,2] + df_10.iloc[i,3])/3
df_10_diff_1 = pd.DataFrame(columns=['diff 1'])
for i in range(24):
    df_10_diff_1 = df_10_diff_1.append(pd.DataFrame({'diff 1':[abs(df_10.iloc[i,1] - df_10.iloc[i,4])/(df_10.iloc[i,1] + df_10.iloc[i,4])]}),ignore_index = True)

df_10_diff_2 = pd.DataFrame(columns=['diff 2'])
for i in range(24):
    df_10_diff_2 = df_10_diff_2.append(pd.DataFrame({'diff 2':[abs(df_10.iloc[i,2] - df_10.iloc[i,4])/(df_10.iloc[i,2] + df_10.iloc[i,4])]}),ignore_index = True)

df_10_diff_3 = pd.DataFrame(columns=['diff 3'])
for i in range(24):
    df_10_diff_3 = df_10_diff_3.append(pd.DataFrame({'diff 3':[abs(df_10.iloc[i,3] - df_10.iloc[i,4])/(df_10.iloc[i,3] + df_10.iloc[i,4])]}),ignore_index = True)
        
df_merge_10_diff_1 = pd.concat([df_10_diff_1, df_10_diff_2],axis = 1)
df_merge_10_diff_2 = pd.concat([df_merge_10_diff_1, df_10_diff_3],axis = 1)
df_10['diff']=0
for i in range(24):
    df_10['diff'][i] = df_merge_10_diff_2.iloc[i,0] + df_merge_10_diff_2.iloc[i,1] + df_merge_10_diff_2.iloc[i,2]
df_10.sort_values("diff",inplace=True,ignore_index = True, ascending = False)
df_10.head(10)

,type,negative_click_rate,neutral_click_rate,positive_click_rate,average_clickrate,diff
0,pressure cooker,0.000000,0.864407,0.714286,0.526231,1.394775
1,vitamin,0.835294,0.814815,0.186667,0.612259,0.828721
2,refrigerator,0.122222,0.388889,0.347368,0.286160,0.650228
3,car,0.391892,0.581699,0.156627,0.376739,0.646248
4,dryer,0.323699,0.734568,0.308642,0.455636,0.595981
5,makeup,0.112676,0.384615,0.245283,0.247525,0.595787
6,pants,0.835821,0.818182,0.333333,0.662445,0.551407
7,furniture,0.825000,0.578947,0.279070,0.561006,0.541818
8,blender,0.742739,0.337963,0.378788,0.486496,0.513093
9,jeans,0.264822,0.609756,0.509881,0.461487,0.459003


In [59]:
df_13 = df_real[['product_type','product', 'Sentiment', 'clickORnot']]
df_13.groupby(['product_type','product', 'Sentiment']).apply(lambda x: x.sum()/len(x))

clickORnot
product_type       product                    Sentiment            
blender            Hamilton Beach blender     negative     0.671642
                                              neutral      0.353846
                                              positive     0.216216
                   NemoK blender              negative     0.978261
                                              neutral      0.325843
                                              positive     0.317460
                   Vitamix blender            negative     0.536585
                                              neutral      0.338710
                                              positive     0.639344
car                Ford  sedan                negative     0.013158
                                              neutral      0.125000
                                              positive     0.265823
                   Tesla                      negative     0.791667
                                              neutral      0.987654
                                              positive     0.057471
coffee             Lavazza Coffee             negative     0.840909
                                              neutral      0.447368
                                              positive     0.342857
                   Starbucks Coffee           negative     0.300000
                                              neutral      0.323529
                                              positive     0.207547
computer           Apple computer             negative     0.700000
                                              neutral      0.923077
                                              positive     0.764706
                   Apple laptop               negative     0.153846
                                              neutral      0.725000
                                              positive     0.772727
                   Dell computer              negative     0.830986
                                              neutral      0.382353
                                              positive     0.719512
                   Dell laptop                negative     0.145299
                                              neutral      0.357143
                                              positive     0.452830
                   Lenova laptop              negative     0.750000
                                              neutral      0.718310
                                              positive     0.419355
dryer              LG dryer                   negative     0.680851
                                              neutral      0.791667
                                              positive     0.395349
                   Maytag dryer               negative     0.117647
                                              neutral      0.704918
                                              positive     0.256757
                   Samsung dryer              negative     0.275862
                                              neutral      0.716981
                                              positive     0.311111
elliptical trainer NordicTrack elliptical     negative     0.491228
                                              neutral      0.737705
                                              positive     0.344828
face cream         Clinique moisturizer       negative     0.902778
                                              neutral      0.930556
                                              positive     0.583333
furniture          Broyhill recliner          negative     0.814286
                                              neutral      0.562500
                                              positive     0.242857
                   Ikea sofa                  negative     0.840000
                                              neutral      0.594203
                                              positive     0.322034
jeans              Cougar jeans              

# <center> Answer to 13(b)

## A recommendation you would make for a product type would also apply to each product under that type.---- Prof. Pradhan

- For a specific product type and sentiment, we can claculate the click rate for each gender type
    - I define the "degree of difference" as:
## <center>degree of difference $= \frac{|clickrate\ (gender\ a)\ -\ average\ clickrate|}{clickrate\ (gender\ a)\ +\ average\ clickrate} $</center>
- And then, I pick the top ten product type. And  I choose just one product for each product type. 

### TOP 10
- Centrum MultiVitamins - ads for Centrum MultiVitamins are most likely to produce clicks in positive context by viewers of female - vitamin


- NordicTrack elliptical - ads for NordicTrack elliptical are most likely to produce clicks in positive context by viewers of male - elliptical trainer


- Docker pants	- ads for Docker pants are most likely to produce clicks in positive context by viewers of female - pants


- Remington shaver - ads for Remington shaver are most likely to produce clicks in positive context by viewers of male - shaver


- NordicTrack rower - ads for NordicTrack rower are most likely to produce clicks in positive context by viewers of female - rowing machine


- Maytag refrigerator - ads for Maytag refrigerator are most likely to produce clicks in positive context by viewers of female - refrigerator	


- Samsung dryer - ads for Samsung dryer are most likely to produce clicks in positive context by viewers of male - dryer


- Covergirl makeup - ads for Covergirl makeup are most likely to produce clicks in positive context by viewers of female - makeup


- Tesla - ads for Tesla are most likely to produce clicks in positive context by viewers of male - car


- Apple iPad - ads for Apple iPad are most likely to produce clicks in positive context by viewers of female - tablet 

In [60]:
df_merge_gender['diff'] = 0
for i in range(24):
    df_merge_gender['diff'][i] = abs(df_merge_gender.iloc[i,2] - df_merge_gender.iloc[i,3])/(df_merge_gender.iloc[i,2] + df_merge_gender.iloc[i,3]+0.00001)
df_merge_gender.sort_values("diff",inplace=True,ignore_index = True, ascending = False)   
df_merge_gender.head(10)

,type,Sentiment,click rate (male),click rate (female),diff
0,vitamin,positive,0.108108,0.263158,0.417613
1,elliptical trainer,positive,0.424242,0.240000,0.277368
2,pants,positive,0.238095,0.416667,0.272723
3,shaver,positive,0.518519,0.312500,0.247908
4,rowing machine,positive,0.218750,0.357143,0.240306
5,refrigerator,positive,0.250000,0.406780,0.238706
6,dryer,positive,0.372093,0.236842,0.222107
7,makeup,positive,0.205882,0.315789,0.210678
8,car,positive,0.183908,0.126582,0.184624
9,tablet,positive,0.428571,0.568182,0.140064


In [61]:
df_13b = df_real[['product_type','product', 'Sentiment', 'gender','clickORnot']]
df_13b.groupby(['product_type','product', 'Sentiment','gender']).apply(lambda x: x.sum()/len(x))

clickORnot
product_type       product                    Sentiment gender            
blender            Hamilton Beach blender     negative  female    0.645161
                                                        male      0.694444
                                              neutral   female    0.425000
                                                        male      0.240000
                                              positive  female    0.189189
                                                        male      0.243243
                   NemoK blender              negative  female    0.948718
                                                        male      1.000000
                                              neutral   female    0.352941
                                                        male      0.289474
                                              positive  female    0.242424
                                                        male      0.400000
                   Vitamix blender            negative  female    0.511111
                                                        male      0.567568
                                              neutral   female    0.344828
                                                        male      0.333333
                                              positive  female    0.694444
                                                        male      0.560000
car                Ford  sedan                negative  female    0.000000
                                                        male      0.024390
                                              neutral   female    0.121951
                                                        male      0.129032
                                              positive  female    0.230769
                                                        male      0.300000
                   Tesla                      negative  female    0.827586
                                                        male      0.767442
                                              neutral   female    1.000000
                                                        male      0.978261
                                              positive  female    0.025000
                                                        male      0.085106
coffee             Lavazza Coffee             negative  female    0.900000
                                                        male      0.791667
                                              neutral   female    0.347826
                                                        male      0.600000
                                              positive  female    0.428571
                                                        male      0.285714
                   Starbucks Coffee           negative  female    0.326087
                                                        male      0.277778
                                              neutral   female    0.400000
                                                        male      0.250000
                                              positive  female    0.224490
                                                        male      0.192982
computer           Apple computer             negative  female    0.666667
                                                        male      0.729730
                                              neutral   female    0.937500
                                                        male      0.909091
                                              positive  female    0.800000
                                                        male      0.727273
                   Apple laptop               negative  female    0.250000
                                                        male      0.000000
                                              neutral   female    0.777778
                                                        male      0.681818
              

# <center> Answer to 13(c)

## A recommendation you would make for a product type would also apply to each product under that type.---- Prof. Pradhan

- For a specific product type and sentiment, we can claculate the click rate for each gender type
    - I define the "degree of difference" as: $i = 1,2,3,4$
## <center>degree of difference $= \frac{|clickrate\ (age\ i)\ -\ average\ clickrate|}{clickrate\ (age\ i)\ +\ average\ clickrate} $</center>
- And then, for each product type, I add up the degree of difference of the four types of age and pick the top ten product type. And  I choose just one product for each product type. 

### TOP 10
- NordicTrack rower - ads for NordicTrack rower are most likely to produce clicks in positive context by viewers of young - rowing machine


- Centrum MultiVitamins - ads for Centrum MultiVitamins are most likely to produce clicks in positive context by viewers of young - vitamin


- Covergirl makeup - ads for Covergirl makeup are most likely to produce clicks in positive context by viewers of young - makeup


- Ford sedan - ads for Ford sedan are most likely to produce clicks in positive context by viewers of senior - car


- Lavazza Coffee - ads for Lavazza Coffee are most likely to produce clicks in positive context by viewers of middle-age - coffee


- Maytag refrigerator - ads for Maytag refrigerator are most likely to produce clicks in negative context by viewers of senior - refrigerator


- Broyhill recliner - ads for Broyhill recliner are most likely to produce clicks in positive context by viewers of young - furniture


- Covergirl makeup - ads for Covergirl makeup are most likely to produce clicks in neutral context by viewers of senior - makeup


- bose speakers - ads for bose speakers are most likely to produce clicks in positive context by viewers of young - speakers


- Kaai handbags - ads for Kaai handbags are most likely to produce clicks in negative context by viewers of juvenile - women's purse

In [62]:
def diff(s1, s2):
    return abs(s1-s2)/(s1+s2)
df_merge_age['diff'] = 0
for i in range(72):
    df_merge_age['diff'][i] = diff(df_merge_age.iloc[i,2],df_merge_age.iloc[i,6])+diff(df_merge_age.iloc[i,3],df_merge_age.iloc[i,6])+diff(df_merge_age.iloc[i,4],df_merge_age.iloc[i,6])+diff(df_merge_age.iloc[i,5],df_merge_age.iloc[i,6])
df_merge_age.sort_values("diff",inplace=True,ignore_index = True, ascending = False) 
df_merge_age.head(15)

,type,Sentiment,click rate1,click rate2,click rate3,click rate4,average_clickrate,significant,diff
0,rowing machine,positive,0.050000,0.714286,0.409091,0.111111,0.321122,yes,1.716610
1,vitamin,positive,0.000000,0.347826,0.125000,0.210526,0.170838,yes,1.600250
2,rowing machine,neutral,0.000000,0.142857,0.176471,0.250000,0.142332,yes,1.383357
3,makeup,positive,0.090909,0.428571,0.133333,0.307692,0.240127,yes,1.141860
4,car,positive,0.050000,0.187500,0.106383,0.322581,0.166616,yes,1.136781
5,rowing machine,negative,0.190476,0.333333,0.050000,0.333333,0.226786,yes,1.106175
6,coffee,positive,0.048780,0.290323,0.361111,0.303030,0.250811,yes,1.021907
7,refrigerator,negative,0.166667,0.040000,0.117647,0.166667,0.122745,yes,0.833164
8,furniture,positive,0.121212,0.405405,0.343750,0.222222,0.273147,yes,0.797433
9,makeup,negative,0.100000,0.058824,0.200000,0.105263,0.116022,yes,0.715657


In [63]:
df_13c = df_real[['product_type','product', 'Sentiment', 'age_group','clickORnot']]
df_13c.groupby(['product_type','product', 'Sentiment','age_group']).apply(lambda x: x.sum()/len(x))

clickORnot
product_type       product                    Sentiment age_group             
blender            Hamilton Beach blender     negative  juvenile      0.533333
                                                        middle-age    0.650000
                                                        senior        0.750000
                                                        young         0.750000
                                              neutral   juvenile      0.153846
                                                        middle-age    0.352941
                                                        senior        0.500000
                                                        young         0.368421
                                              positive  juvenile      0.235294
                                                        middle-age    0.136364
                                                        senior        0.235294
                                                        young         0.277778
                   NemoK blender              negative  juvenile      0.952381
                                                        middle-age    1.000000
                                                        senior        0.954545
                                                        young         1.000000
                                              neutral   juvenile      0.300000
                                                        middle-age    0.363636
                                                        senior        0.363636
                                                        young         0.280000
                                              positive  juvenile      0.210526
                                                        middle-age    0.333333
                                                        senior        0.333333
                                                        young         0.411765
                   Vitamix blender            negative  juvenile      0.708333
                                                        middle-age    0.545455
                                                        senior        0.235294
                                                        young         0.578947
                                              neutral   juvenile      0.444444
                                                        middle-age    0.090909
                                                        senior        0.411765
                                                        young         0.312500
                                              positive  juvenile      0.571429
                                                        middle-age    0.538462
                                                        senior        0.705882
                                                        young         0.705882
car                Ford  sedan                negative  juvenile      0.000000
                                                        middle-age    0.000000
                                                        senior        0.052632
                                                        young         0.000000
                                              neutral   juvenile      0.000000
                                                        middle-age    0.000000
                                                        senior        0.400000
                                                        young         0.050000
                                              positive  juvenile      0.100000
                                                        middle-age    0.227273
                                                        senior        0.562500
                                                        young         0.238095
                   Tesla                      negative  juvenile      0.812500
                                       